# Лабораторна робота №4

Оскільки у попередній лабораторній ми використовували детектор та класифікатор, то за завдання цієї лабораторної роботи ми взяли пункт Б. Через те, що в умові задачі не було зазначено щоб кожен член команди використовував свої предмети на розгляді, ми вирішили написати одну програму, для розпізнавання одного маркера - обкладинки книжки "Граф Монте-Крісто", та подальшою заміною його на обкладинку книжки "Пригоди бравого вояка Швейка". Але ми використовували два класифікатора KAZE та ORB, як і в попередніх лабораторних, щоб в кінці порівняти іх роботу. Ми, Аблець Андрій та Володимир Возняк, також аргументуємо написання однієї програми тим, що це є легшим для сприйняття викладача, оскільки двічі дивитись приблизно однакові програми не є оптимальним. 

In [1]:
import cv2
import numpy as np
import os
import glob

In [15]:
# replaceable_img - фотографія, яку ми хочемо замінити
# original_vid - відео, що містить replaceable_img, яку ми будемо замінити
# replacement_img - цією фотографією ми замінюємо replaceable_img


replaceable_img = cv2.imread('replaceable_image.jpg')
original_vid = cv2.VideoCapture('original_video.mp4')
replacement_img = cv2.imread('replacement_image.jpg')
# replacement_vid = cv2.VideoCapture('replacement_video.mp4')

success_original, original_vid_frame = original_vid.read()
# success_replacement, replacement_vid_frame = replacement_vid.read()

height, width, channels = replaceable_img.shape

# replacement_vid_frame = cv2.resize(replacement_vid_frame,(width, height))

# detector  = cv2.KAZE_create()
# matcher = cv2.BFMatcher(cv2.NORM_L1)
detector  = cv2.ORB_create(nfeatures=3000)
matcher = cv2.BFMatcher()
kp_original, des_original = detector.detectAndCompute(replaceable_img, None)
replaceable_img = cv2.drawKeypoints(replaceable_img,kp_original,None)

## Заміна відео

Розділимо наше оригінальне відео на фрейми

In [12]:
import cv2
vidcap = cv2.VideoCapture('original_video.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("original_video_frames/%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  count += 1

Для кожного фрейму проведимо заміну книжок та зберігаємо результати у папку

In [11]:
img_names = [i for i in range(len(glob.glob('original_video_frames/*.jpg')))]

for img_index in img_names:

    photo_dir = 'original_video_frames' + '\\' + str(img_index) + '.jpg'
    image_frame  = cv2.imread(photo_dir,1)
#     kp, des = detector.detectAndCompute(image, None)
    
#     matcher = cv2.BFMatcher(cv2.NORM_L1)
    matcher = cv2.BFMatcher()

#     success_image, image_frame = image.read()
    image_frame_copy = image_frame.copy( )
    kp_image, des_image = detector.detectAndCompute(image_frame, None)
    matches = matcher.knnMatch(des_original, des_image, k=2)
    good = []
    
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)
            
    print(len(good))
    if len(good)>35:

        matching_result = cv2.drawMatches(replaceable_img, kp_original, image_frame, kp_image, good, None,flags=2)

        original_points = np.float32([kp_original[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        image_points = np.float32([kp_image[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        matrix, mask = cv2.findHomography(original_points,image_points,cv2.RANSAC)

        corner_points = np.float32([[0,0],[0,height],[width,height],[width,0]]).reshape(-1,1,2)
        
        try:
            destination_points = cv2.perspectiveTransform(corner_points,matrix)

            polygon_img = cv2.polylines(image_frame, [np.int32(destination_points)],True,(100,100,0),5)

            image_warp = cv2.warpPerspective(replacement_img, matrix, ((image_frame.shape[1],image_frame.shape[0])))

            black_frame = np.zeros((image_frame.shape[0],image_frame.shape[1]), np.uint8)
            cv2.fillPoly(black_frame,[np.int32(destination_points)],(255,255,255))
            inv_black_frame = cv2.bitwise_not(black_frame)
            image_frame_copy = cv2.bitwise_and(image_frame_copy,image_frame_copy,mask=inv_black_frame)

            result_image = cv2.bitwise_or(image_warp, image_frame_copy)
            
            cv2.imwrite("ORB_replaced_video_frames/%d.jpg" % img_index, result_image)
        except:
            cv2.imwrite("ORB_replaced_video_frames/%d.jpg" % img_index, image_frame)
            continue
            
    else:
        cv2.imwrite("ORB_replaced_video_frames/%d.jpg" % img_index, image_frame)


Об'єднуємо змінені фотографії в єдине відео

In [12]:
img_names = [i for i in range(len(glob.glob('ORB_replaced_video_frames/*.jpg')))]
img_array = [i for i in range(len(glob.glob('ORB_replaced_video_frames/*.jpg')))]

for img_index in img_names:
    photo_dir = 'ORB_replaced_video_frames' + '\\' + str(img_index) + '.jpg'
    image_frame  = cv2.imread(photo_dir,1)
    img_array[img_index] = image_frame
 

height, width, layers = img_array[0].shape
size = (width,height)
out = cv2.VideoWriter('ORB_result_video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Заміна у реальному часі

In [17]:
webcam = cv2.VideoCapture(0)
img_index = 0
# replacement_img = cv2.resize(replacement_img,(640, 480))
# replaceable_img = cv2.resize(replaceable_img,(640, 480))

while True:
#     matcher = cv2.BFMatcher(cv2.NORM_L1)
    matcher = cv2.BFMatcher()

    success_webcam, webcam_frame = webcam.read()
    webcam_frame_copy = webcam_frame.copy( )
    kp_webcam, des_webcam = detector.detectAndCompute(webcam_frame, None)
#     webcam_frame = cv2.drawKeypoints(webcam_frame,kp,None)
    matches = matcher.knnMatch(des_original, des_webcam, k=2)
    good = []
    
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            good.append(m)
            
    
    if len(good)>35:

        matching_result = cv2.drawMatches(replaceable_img, kp_original, webcam_frame, kp_webcam, good, None,flags=2)

        original_points = np.float32([kp_original[m.queryIdx].pt for m in good]).reshape(-1,1,2)
        replacement_points = np.float32([kp_webcam[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        matrix, mask = cv2.findHomography(original_points,replacement_points,cv2.RANSAC)

        corner_points = np.float32([[0,0],[0,height],[width,height],[width,0]]).reshape(-1,1,2)
        
        try:
            destination_points = cv2.perspectiveTransform(corner_points,matrix)

            polygon_img = cv2.polylines(webcam_frame, [np.int32(destination_points)],True,(100,100,0),5)
            cv2.imshow('polygon_img', polygon_img)
            cv2.imwrite("real_time_result/ORB/ORB_polygon_frames/%d.jpg" % img_index, polygon_img)

            image_warp = cv2.warpPerspective(replacement_img, matrix, ((webcam_frame.shape[1],webcam_frame.shape[0])))

            black_frame = np.zeros((webcam_frame.shape[0],webcam_frame.shape[1]), np.uint8)
            cv2.fillPoly(black_frame,[np.int32(destination_points)],(255,255,255))
            
            inv_black_frame = cv2.bitwise_not(black_frame)
            webcam_frame_copy = cv2.bitwise_and(webcam_frame_copy,webcam_frame_copy,mask=inv_black_frame)

            result_image = cv2.bitwise_or(image_warp, webcam_frame_copy)
            cv2.imshow('result_image',result_image)
            cv2.imwrite("real_time_result/ORB/ORB_result_frames/%d.jpg" % img_index, result_image)
            img_index += 1
            
        except:
            cv2.imshow('polygon_img', webcam_frame)
            cv2.imwrite("real_time_result/ORB/ORB_polygon_frames/%d.jpg" % img_index, webcam_frame)

            cv2.imshow('result_image',webcam_frame)
            cv2.imwrite("real_time_result/ORB/ORB_result_frames/%d.jpg" % img_index, webcam_frame)
            continue
            
    else:
        cv2.imshow('polygon_img', webcam_frame)
        cv2.imwrite("real_time_result/ORB/ORB_polygon_frames/%d.jpg" % img_index, webcam_frame)

        cv2.imshow('result_image',webcam_frame)
        cv2.imwrite("real_time_result/ORB/ORB_result_frames/%d.jpg" % img_index, webcam_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
        
webcam.release() 
cv2.destroyAllWindows() 


Об'єднуємо змінені фотографії в два відео

In [4]:
img_names = [i for i in range(len(glob.glob('real_time_result/ORB/ORB_result_frames/*.jpg')))]
img_array_results = [i for i in range(len(glob.glob('real_time_result/ORB/ORB_result_frames/*.jpg')))]
img_array_polygons = [i for i in range(len(glob.glob('real_time_result/ORB/ORB_result_frames/*.jpg')))]


for img_index in img_names:
    photo_dir_results = 'real_time_result\\ORB\\ORB_result_frames' + '\\' + str(img_index) + '.jpg'
    photo_dir_polygons = 'real_time_result\\ORB\\ORB_polygon_frames' + '\\' + str(img_index) + '.jpg'
    
    image_frame_results  = cv2.imread(photo_dir_results,1)
    image_frame_polygons  = cv2.imread(photo_dir_polygons,1)
    
    img_array_results[img_index] = image_frame_results
    img_array_polygons[img_index] = image_frame_polygons
 

height, width, layers = img_array_results[0].shape
size = (width,height)

out_results = cv2.VideoWriter('real_time_result\\ORB\\real_time_result_video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
out_polygons = cv2.VideoWriter('real_time_result\\ORB\\real_time_polygons_video.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array_results)):
    out_results.write(img_array_results[i])
    out_polygons.write(img_array_polygons[i])
out_results.release()
out_polygons.release()

## Висновок

Насправді, результат програми не є неочікуваним, оскільки за плечами в нас 2 роботи з порівнянням класифікаторів KAZE та ORB. Як і в попередніх роботах, KAZE краще розпізнав об'єкт ніж ORB. Це дуже добре видно на початку відео, коли книга лежить доволі далеко від камери. В цьому випадку ORB не може розпізнати книгу, до моменту, коли камера приблизиться до неї. Після наближення камери результат роботи класифікаторів не сильно відрізняється. І це при тому, що ми задали кількість фіч у ORB = 3000 (у попередніх лаботарних ми використовували 500). Якщо ж тут написати 500, то результат буде набагато гірше. І це ще одне підтвердження того, що ORB дуже погано справляється з нечіткими фотографіями. Але якщо порівнювати класифікатори по часу роботи, то, як і в попередніх лабах KAZE сильно поступається ORB. Ще нами була реалізована заміна фотографій у реальному часі, де теж KAZE веде себе дещо краще за ORB, але це не так добре видно, як при роботі з відео. В цій задачі обидва класифікатори упоралися дещо гірше  ніж у роботі з відео. Ми думаємо, що це із-за того, що роздільна якість веб-камери доволі мала(480х640).